<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

In [ ]:
# search file paths
from glob import glob


img_paths_train = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png')) # *: all
img_paths_val = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png'))

In [ ]:
# number of images
len(img_paths_train), len(img_paths_val)

In [ ]:
import torch
import imageio.v2 as imageio
from PIL import Image
import numpy as np


IMG_SIZE = 224
class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size, transform):
        self.img_paths = img_paths
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = imageio.imread(path) # read img
        img_pil = Image.fromarray(img)
        img = img_pil.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)
        img = np.array(img)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB

        # transform img
        img = self.transform(img)

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])
        cls_idx = torch.tensor(cls_idx, dtype=torch.long)

        return img, cls_idx

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights


# Preprocess Transform
transform =  ResNet50_Weights.IMAGENET1K_V2.transforms()

# Build Dataset
IMG_SIZE = 224 # ResNet-50 typically takes input images of size 224x224 pixels.
train_ds = PretrainDataset(img_paths_train, IMG_SIZE, transform)
val_ds = PretrainDataset(img_paths_val, IMG_SIZE, transform)

In [ ]:
transform

In [ ]:
# Build DataLoaders
BATCH_SIZE = 150
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE)

#### Build Model (https://pytorch.org/vision/stable/models.html#classification)

In [ ]:
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
print(model)

In [ ]:
# replace classifier
num_features = model.fc.in_features # len of feature vectors

# # Freeze model
# for param in model.parameters():
#     param.requires_grad = False

# Replace classifier
model.fc = nn.Linear(num_features, 50)
print(model.fc)

In [ ]:
summary(model.to(device), (3, 224, 224))

#### Train Model

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode.
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
import matplotlib.pyplot as plt


def train(pretrained, freeze=False):
    print(f"pretrained weights: {pretrained}, freeze: {freeze}")
    # Model
    if pretrained:
        model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    else:
        model = models.resnet50(weights=None)

    if freeze:
        # Freeze model
        for param in model.parameters():
            param.requires_grad = False
    # Replace classifier
    num_features = model.fc.in_features # len of feature vectors
    model.fc = nn.Linear(num_features, 50)

    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    # lower learning rate for finetuning
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=1e-4)
    # schedule = ReduceLROnplateau(optimizer, 'min')

    EPOCHS = 10
    logs = {
        'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
    }
    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = train_epoch(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = test_epoch(val_loader, model, loss_fn)

        # # Note that step should be called after test_epoch()
        # schedule.step(val_loss)

        print(f'EPOCH: {epoch:04d} \
        train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
        val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

        logs['train_loss'].append(train_loss)
        logs['train_acc'].append(train_acc)
        logs['val_loss'].append(val_loss)
        logs['val_acc'].append(val_acc)

    # plot result
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.plot(logs['train_loss'])
    plt.plot(logs['val_loss'])
    plt.subplot(1, 2, 2)
    plt.title('Acc.')
    plt.plot(logs['train_acc'])
    plt.plot(logs['val_acc'])
    plt.show()

    # Save model
    torch.save(model, MODEL_PATH)

In [ ]:
train(pretrained=False)
# transfer learning
train(pretrained=True)
train(pretrained=True, freeze=True)